In [7]:
from newsapi import NewsApiClient
import requests
from PIL import Image, ImageOps
from marxify import Marxify
import tweepy
import random
import time

In [12]:
# Prepare APIs

# NewsAPI
newsApi = NewsApiClient(api_key='bbba33d91ef64b5cafeea9be97da9538')
country = 'us' 

# Twitter API
# I have regenerated these so they won't work
consumer_key = 'WrPTmBaJESVs7eLfwkiVaqQ7U'
consumer_secret = 'MQmk7QllutWAtVMA9GWDenk4Ta2npiBQY8qFiyyim7ksQHhSP6'
access_token = '3264530906-gJCKvK5i0sqSZaWTkRvUZVkpgYNGEAf5fIA2oP8'
access_token_secret = 'fWvCRNVs3D3LLxlujVtw6DJNaY37dHxRMsLFlSuEHQr4l'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

twitterApi = tweepy.API(auth)

# picture file name
pic_filename = 'pic.png'

In [13]:
def uploadCommunistTweet(newsApi, twitterApi, modelChoice='communist'):
    # Get latest news and picture
    headlines = newsApi.get_top_headlines(country=country)
    articles = sorted([r for r in headlines['articles'] if r['content'] is not None], key=lambda r: r['content'])
    selectIdx = random.randrange(min(len(articles), 10))
    articleJson = articles[selectIdx]
    imageUrl = articleJson['urlToImage']
    image = Image.open(requests.get(imageUrl, stream=True).raw).convert(mode='RGBA')
    image = image.resize((800,450))

    # overlay communist symbol
    communistSymbol = Image.open('assets/hammer_sickle_and_star.png').resize((180,200))
    image.alpha_composite(communistSymbol, (10,10))
    image.save(pic_filename)
    
    # train model
    model = Marxify(articleJson['content'][:-15])
    
    # get status text
    modelsDict = {
        'communist': model.makeCommunistSentence,
        'marx': model.marxSays,
        'engels': model.engelsSays,
        'che': model.cheSays
    }
    status = modelsDict[modelChoice]()
    status += ' #%s #%s #%s #marxovchain' % (model._keywords[0], model._keywords[1], model._keywords[2])
    
    # upload to Twitter
    twitterApi.update_with_media(status=status, filename=pic_filename)

In [18]:
modelChoice = random.choice(['communist', 'marx', 'engels', 'che'])
uploadCommunistTweet(newsApi, twitterApi, modelChoice=modelChoice)